In [2]:
import pandas as pd
df=pd.read_excel("/content/Unstructured_Financial_Data.xlsx")
df.head()

,Transaction Date,Product,Region,Department,Category,Revenue,Cost,Expense Type,Expense Amount
0,10/03/2024,monitor,East,Finance,Electronics,6570.27,3801.96,R&D,520.78
1,NaN,Monitor,NaN,Operations,elec,6842.89,1585.93,NaN,1846.74
2,05/30/2024,laptop,south,Operations,NaN,9228.08,1377.45,Marketing,486.02
3,NaN,Lap-top,NaN,NaN,Mobile,5222.43,5966.81,Marketing,318.40
4,12/23/2024,laptop,NaN,Sales,Electronics,6930.94,4906.67,R&D,1035.10


In [4]:
df['Transaction Date']=pd.to_datetime(df['Transaction Date'],errors='coerce')
df[df['Transaction Date'].isnull()]

,Transaction Date,Product,Region,Department,Category,Revenue,Cost,Expense Type,Expense Amount
1,NaT,Monitor,NaN,Operations,elec,6842.89,1585.93,NaN,1846.74
3,NaT,Lap-top,NaN,NaN,Mobile,5222.43,5966.81,Marketing,318.40
6,NaT,Phone,NaN,ops,Mobile,3129.22,3759.12,R&D,583.27
9,NaT,NaN,WEST,Finance,elec,8723.73,4239.00,Marketing,1764.21
12,NaT,NaN,WEST,Sales,Mobile,6549.69,NaN,Marketing,1732.86
...,...,...,...,...,...,...,...,...,...
141,NaT,moniTor,WEST,NaN,elec,7641.37,585.94,NaN,692.11
142,NaT,laptop,south,sales,Electronics,4512.90,5633.95,Salaries,1347.85
143,NaT,Monitor,south,NaN,Mobile,5071.00,NaN,Salaries,787.30
146,NaT,NaN,NaN,sales,NaN,3610.32,4198.43,NaN,1306.58


In [5]:
df['Product'].unique()

array(['monitor', 'Monitor', 'laptop', 'Lap-top', nan, 'Phone', 'Laptop',
       'Tab', 'moniTor', 'Tablet'], dtype=object)

In [7]:
!pip install thefuzz
import pandas as pd
from thefuzz import fuzz
from thefuzz import process

In [11]:
def generate_mapping(column_values,threshold=85):
  mapping={}
  already_mapped=set()
  for val in column_values:
    if pd.isnull(val) or val in already_mapped:
      continue
    matches= process.extract(val,column_values,limit=None,scorer=fuzz.token_sort_ratio)
    close_matches=[match[0]for match in matches if match[1]>= threshold]
    for match in close_matches:
      mapping[match]= val
      already_mapped.add(match)
  return mapping

def clean_column(df,column_name,mapping):
  df[column_name]=df[column_name].replace(mapping)
  return df

In [12]:
raw_vals=df['Product'].dropna().astype(str).str.strip().str.lower().unique()
product_mapping=generate_mapping(raw_vals,threshold=85)
print("Suggested Mapping:")
for k, v in product_mapping.items():
    print(f"{k} → {v}")

df['Product'] = df['Product'].str.strip().str.lower()
df = clean_column(df, 'Product', product_mapping)


Suggested Mapping:
monitor → monitor
laptop → laptop
lap-top → laptop
phone → phone
tab → tab
tablet → tablet


In [13]:
def clean_categorical_column(df, colname, threshold=85, print_mapping=False):
    df[colname] = df[colname].astype(str).str.strip().str.lower()
    uniques = df[colname].dropna().unique()
    mapping = generate_mapping(uniques, threshold=threshold)
    if print_mapping:
        print(f"\n Mapping for '{colname}':")
        for k, v in mapping.items():
            if k != v:
                print(f"'{k}' → '{v}'")
    df = clean_column(df, colname, mapping)
    return df

categorical_columns = df.select_dtypes(include='object').columns.tolist()
for col in categorical_columns:
    df = clean_categorical_column(df, col, threshold=85, print_mapping=True)




 Mapping for 'Product':

 Mapping for 'Region':

 Mapping for 'Department':

 Mapping for 'Category':
'mobiles' → 'mobile'

 Mapping for 'Expense Type':


In [14]:
df['Profit'] = df['Revenue'] - df['Cost']
df['Profit Margin'] = df.apply(lambda row: row['Profit'] / row['Revenue']
                               if row['Revenue'] and row['Revenue'] != 0 else None, axis=1)

df['Year'] = df['Transaction Date'].dt.year
df['Month'] = df['Transaction Date'].dt.strftime('%B')
df['Month-Year'] = df['Transaction Date'].dt.strftime('%b %Y')


In [15]:
df.to_excel('cleaned_financial_data.xlsx', index=False)
from google.colab import files
files.download('cleaned_financial_data.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>